In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
df = pd.read_csv("../dataset/PatientDetails_Classification.csv")

X = df.iloc[:,1:-1]
y = df.iloc[:,-1]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=56) 
n_splits = 3
indices = np.arange(len(X))
np.random.shuffle(indices)

fold_indices = np.array_split(indices, n_splits)


for i in range(n_splits):
    test_indices = fold_indices[i]
    train_indices = np.concatenate(fold_indices[:i] + fold_indices[i+1:])

    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]


class SVM:
    def __init__(self, learning_rate=0.001,n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X.values):
                condition = y.iloc[idx] * (np.dot(x_i, self.w) + self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.w)
                else:
                    self.w -= self.lr * (2*self.w - np.dot(x_i, y.iloc[idx]))
                    self.b -= self.lr * y.iloc[idx]

    def predict(self, X):
        prediction = np.dot(X, self.w) + self.b
        return np.sign(prediction).astype(int)
clf = SVM()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

accuracy=accuracy_score(y_test,predictions)
print(accuracy)

0.625


In [2]:
from sklearn.metrics import confusion_matrix
predictions_test=clf.predict(X_test.values)
cm = confusion_matrix(y_test, predictions_test)
tp = cm[0, 0]
fp = cm[0, 1]
fn = cm[1, 0]
tn = cm[1, 1]
accuracy=(tp+tn)/(tp+fp+fn+tn)
print(accuracy)

0.625
